In [1]:
import numpy as np

# HW 8
## Primal vs Dual Problem

- The SVM **primal** problem minimizes 0.5w<sup>T</sup>w subject to y<sub>n</sub>(w<sup>T</sup>x<sub>n</sub>+b)&ge;1 for n = 1,2,...,N
- Since w is a d-dimensional vector (corresponding to the dimension of x) and we can also vary b, the primal problem involves a quadratic programming problem with **d+1 variables**.